In [2]:
!huggingface-cli whoami

m-aliabbas1


In [3]:
from huggingface_hub import HfApi

user_id = HfApi().whoami()["name"]


In [4]:
from datasets import *
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizerFast


In [5]:
tokenizer_id="bert-tiny-waspak-2023-papia1"

In [6]:
!ls ..

ali.txt			 exp_notebooks			    models
clean_filenames.csv	 miniconda3			    text_col.csv
exp_language_model_work  Miniconda3-latest-Linux-x86_64.sh  tokenizer_papia


In [7]:
df = pd.read_csv('../clean_filenames.csv')

In [8]:
df=df.drop(df.columns[:2],axis=1)

df = df.rename(columns={'data':'text'})

df.to_csv('text_col.csv',index=False)

text_list = list(df['text'].values)

In [9]:
def batch_iterator(batch_size=100):
    for i in tqdm(range(0, len(text_list), batch_size)):
        batch = text_list[i : i + batch_size]
        yield [str(text) for text in batch]

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("prajjwal1/bert-tiny")

In [13]:
tokenizer.vocab_size

30522

In [14]:
vocab_size = 30522

bert_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=vocab_size,max_length=512,model_max_length=512)


Ignored unknown kwargs option max_length
Ignored unknown kwargs option model_max_length


100%|█████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:10<00:00,  4.68it/s]


In [15]:
bert_tokenizer.save_pretrained("tokenizer_tiny_bert_papia")

('tokenizer_tiny_bert_papia/tokenizer_config.json',
 'tokenizer_tiny_bert_papia/special_tokens_map.json',
 'tokenizer_tiny_bert_papia/vocab.txt',
 'tokenizer_tiny_bert_papia/added_tokens.json',
 'tokenizer_tiny_bert_papia/tokenizer.json')

In [16]:
bert_tokenizer.push_to_hub(tokenizer_id)

CommitInfo(commit_url='https://huggingface.co/m-aliabbas1/bert-tiny-waspak-2023-papia1/commit/48c2aeb4fc0a22cd6dc223cf0a396711d4a59585', commit_message='Upload tokenizer', commit_description='', oid='48c2aeb4fc0a22cd6dc223cf0a396711d4a59585', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from transformers import AutoTokenizer
import multiprocessing


tokenizer = AutoTokenizer.from_pretrained(f'{user_id}/{tokenizer_id}')
num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 1000000000000000019884624838656


In [18]:
text_list = [str(text) for text in text_list]

In [19]:
raw_data = {'text':text_list}

In [20]:
num_proc = multiprocessing.cpu_count()

In [21]:
def group_texts(examples):
#     print(examples['text'])
    tokenized_inputs = tokenizer(
       examples['text'], return_special_tokens_mask=True, truncation=True, max_length=512
    )
    return tokenized_inputs

In [22]:
from datasets import Dataset

In [23]:
dataset = Dataset.from_dict(raw_data)

In [24]:
dataset = dataset.train_test_split(
    test_size=0.2)

In [25]:
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)

Map (num_proc=24):   0%|          | 0/3724 [00:00<?, ? examples/s]

Map (num_proc=24):   0%|          | 0/931 [00:00<?, ? examples/s]

In [26]:
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

In [27]:
from transformers import Trainer, TrainingArguments

In [28]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

/home/mohammad/miniconda3/envs/ali_nlp/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [29]:
vocab_size = 30522
max_length = 512

In [34]:
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)
model

Generate config GenerationConfig {
  "_from_model_config": true,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}



BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [35]:
model_config = AutoConfig.from_pretrained('prajjwal1/bert-tiny',max_position_embeddings=max_length)

loading configuration file config.json from cache at /home/mohammad/.cache/huggingface/hub/models--prajjwal1--bert-tiny/snapshots/6f75de8b60a9f8a2fdf7b69cbd86d9e64bcb3837/config.json
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [36]:
model_config

BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [37]:
model = BertForMaskedLM(config=model_config)

Generate config GenerationConfig {
  "_from_model_config": true,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}



In [38]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_aff

In [39]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [40]:
model_path = 'models/'

In [56]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=40, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=30,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=30,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 30
Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [57]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
    
)

In [58]:
# train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3,724
  Num Epochs = 10
  Instantaneous batch size per device = 40
  Training with DataParallel so batch size has been adjusted to: 80
  Total train batch size (w. parallel, distributed & accumulation) = 640
  Gradient Accumulation steps = 8
  Total optimization steps = 50
  Number of trainable parameters = 4,416,698


Step,Training Loss,Validation Loss
30,8.686400,8.629770


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 931
  Batch size = 128
Saving model checkpoint to models/checkpoint-30
Configuration saved in models/checkpoint-30/config.json
Configuration saved in models/checkpoint-30/generation_config.json
Model weights saved in models/checkpoint-30/pytorch_model.bin
/home/mohammad/miniconda3/envs/ali_nlp/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from models/checkpoint-30 (sc

TrainOutput(global_step=50, training_loss=8.652171325683593, metrics={'train_runtime': 446.9944, 'train_samples_per_second': 83.312, 'train_steps_per_second': 0.112, 'total_flos': 43262920949760.0, 'train_loss': 8.652171325683593, 'epoch': 8.51})

In [60]:
model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-30"))

loading configuration file models/checkpoint-30/config.json
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file models/checkpoint-30/pytorch_model.bin
Generate config GenerationConfig {
  "_from_model_config": true,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}

All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMa

In [61]:
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer,device=model.device)

In [62]:
text_list[0]

'club nacional de football miho conoci como nacional ta club mas grandi di futbol di montevidéu uruguay fundá dia 14 di mei 1899 club ta resultado di union entre uruguay athletic montevideo football club uruguay athletic tabata un club di bario la union cual no mester wordo confundi cu uruguay athletic club cu tabata hunga den prome division actualmente nacional ta hunga den liga profesional mas halto na uruguay algun futbolista ku tabata hunga pa nacional ta luis suarez uruguay sebastian abreu uruguay atilio garcia argentina hugo de león uruguay nicolás lodeiro uruguay héctor scarone uruguay julio cesar dely valdéz panama fernando muslera uruguay titulos campeon nashonal liga profesional di uruguay 45 1902 1903 1912 1915 1916 1917 1919 1920 1922 1923 1924 1933 1934 1939 1940 1941 1942 1943 1946 1947 1950 1952 1955 1956 1957 1963 1966 1969 1970 1971 1972 1977 1980 1983 1992 1998 2000 2001 2002 2005 2005 06 2008 09 2010 11 2011 12 2014 15 2016 campeon kontinental sur amerika cup 3 1971 

In [67]:
examples = "club nacional de [MASK] miho conoci como nacional ta club mas grandi di futbol di montevidéu uruguay fundá dia 14 di mei 1899 club ta resultado di union entre uruguay athletic montevideo football club uruguay athletic tabata un club di bario la union cual no mester wordo confundi cu uruguay athletic club cu tabata hunga den prome division actualmente nacional ta hunga den liga profesional mas halto na uruguay algun futbolista ku tabata hunga pa nacional ta luis suarez uruguay sebastian abreu uruguay atilio garcia argentina hugo de león uruguay nicolás lodeiro uruguay héctor scarone uruguay julio cesar dely valdéz panama fernando muslera uruguay titulos campeon nashonal liga profesional di uruguay 45 1902 1903 1912 1915 1916 1917 1919 1920"

In [69]:
fill_mask(examples)

[{'score': 0.0007799027371220291,
  'token': 602,
  'token_str': 'di',
  'sequence': 'club nacional de di miho conoci como nacional ta club mas grandi di futbol di montevideu uruguay funda dia 14 di mei 1899 club ta resultado di union entre uruguay athletic montevideo football club uruguay athletic tabata un club di bario la union cual no mester wordo confundi cu uruguay athletic club cu tabata hunga den prome division actualmente nacional ta hunga den liga profesional mas halto na uruguay algun futbolista ku tabata hunga pa nacional ta luis suarez uruguay sebastian abreu uruguay atilio garcia argentina hugo de leon uruguay nicolas lodeiro uruguay hector scarone uruguay julio cesar dely valdez panama fernando muslera uruguay titulos campeon nashonal liga profesional di uruguay 45 1902 1903 1912 1915 1916 1917 1919 1920'},
 {'score': 0.0005202289903536439,
  'token': 606,
  'token_str': 'ta',
  'sequence': 'club nacional de ta miho conoci como nacional ta club mas grandi di futbol di mo

In [72]:
model.push_to_hub('bert-tiny-waspak-2023-papia1')

Configuration saved in /tmp/tmpa26nfhei/config.json
Configuration saved in /tmp/tmpa26nfhei/generation_config.json
Model weights saved in /tmp/tmpa26nfhei/pytorch_model.bin
Uploading the following files to m-aliabbas1/bert-tiny-waspak-2023-papia1: generation_config.json,config.json,pytorch_model.bin


pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/m-aliabbas1/bert-tiny-waspak-2023-papia1/commit/d868cc937188f111548924dc3473d886446f1da9', commit_message='Upload BertForMaskedLM', commit_description='', oid='d868cc937188f111548924dc3473d886446f1da9', pr_url=None, pr_revision=None, pr_num=None)